In [69]:
import random
import math
from random import choice
import json
import time

In [73]:
with open("input-data.json", "r") as f:
    config_data = json.load(f)


In [60]:
class Patch:
    def __init__(self, index, climate_zone, soil_quality, config):
        self.index = index
        self.climate_zone = climate_zone
        self.soil_quality = soil_quality
        
        # Используем конфигурационные параметры для определения soil_factor
        self.soil_factor = config["SOILS"].get(soil_quality, 0)

        self.grass = config.get("GRASS_START", 0.0)
        self.max_grass = config.get("GRASS_MAX", 1.0)

    def grow_grass(self, grass_speed):
        delta = self.temperature * self.soil_factor * grass_speed
        self.grass = min(self.grass + delta, self.max_grass)


In [75]:
class World:
    def __init__(self, config_data):
        self.config_data = config_data
        self.length = config_data["LENGTH"]
        self.season_length = config_data["SEASON_LENGTH"]
        self.grow_speed = config_data["GROW_SPEED"]

        self.patches = []
        self.cows = []     # Живые коровы
        self.new_cows = [] # Новые потомки

        self.temperature_patterns = config_data["CLIMATES"]  # Получаем климатические зоны из конфига

        self.current_tick = 0
        self.init_patches()

    def init_patches(self):
        for i in range(self.length):
            # Формируем оазис
            if 501 <= i < 3500:
                climate = self.get_climate_for_oasis(i)
                soil = self.get_soil_for_oasis(i)
            else:
                climate = 'Polar'
                soil = 'desert'

            patch = Patch(i, climate, soil, self.config_data)  # Передаем config_data в Patch
            self.patches.append(patch)

    def get_climate_for_oasis(self, cell_number):
        # Оазис с климатами "Equator", "Middle", "Polar" по циклу
        zones = list(self.config_data["CLIMATES"].keys())  # Чтение климатических зон из конфига
        return zones[(cell_number - 501) % len(zones)]

    def get_soil_for_oasis(self, cell_number):
        # Оазис с разными типами почвы
        soil_types = list(self.config_data["SOILS"].keys())  # Чтение типов почвы из конфига
        return soil_types[(cell_number - 501) % len(soil_types)]

    def update_season(self):
        season = (self.current_tick // self.season_length) % 4  # 0: зима, 1: весна, 2: лето, 3: осень
        for patch in self.patches:
            pattern = self.temperature_patterns.get(patch.climate_zone, [1, 1, 1, 1])
            patch.temperature = pattern[season]

    def update(self):
        self.update_season()
        for patch in self.patches:
            patch.grow_grass(self.grow_speed)
        self.current_tick += 1


In [76]:
class Cow:
    def __init__(self, position, birth_tick, config, parents_qty=1):
        self.position = position
        self.energy = config["ATP_START"]
        self.raw_eat = config["RAW_EAT_START"]

        self.size = random.randint(1, 10)
        self.speed = random.randint(1, 10)
        self.metabolism = random.randint(1, 10)
        self.sight = random.randint(1, 10)
        self.herding = random.randint(1, 10)

        self.birth_tick = birth_tick
        self.death_tick = None
        self.T0 = random.randint(1, 10)
        self.parents_qty = parents_qty

        # config-параметры
        self.age_min = config["AGE_MIN"]
        self.age_mult = config["AGE_MULT"]
        self.age_t_weight = config["AGE_T_WEIGHT"]
        self.chance_num = config["CHANCE_NUMERATOR"]
        self.chance_den = config["CHANCE_DENOMINATOR"]
        self.breed_type = config["BREED_TYPE"]
        self.birth_count = config["BIRTH_COUNT"]

        self.config = config  # сохраняем для потомства

    def move(self, world):
        min_pos = max(0, self.position - self.sight)
        max_pos = min(world.length, self.position + self.sight + 1)

        best_pos = self.position
        best_grass = world.patches[self.position].grass

        for pos in range(min_pos, max_pos):
            patch = world.patches[pos]
            cow_count = sum(1 for cow in world.cows if cow.position == pos and cow != self)
            if cow_count > self.herding:
                continue
            if patch.grass > best_grass:
                best_grass = patch.grass
                best_pos = pos

        if best_pos != self.position:
            distance = abs(best_pos - self.position)
            if distance <= self.speed:
                self.position = best_pos

    def eat(self, patch):
        eat_amount = min(self.size * 0.1, patch.grass)
        patch.grass -= eat_amount
        self.energy += eat_amount

    def update(self, current_tick, world):
        age = current_tick - self.birth_tick
        self.energy -= self.metabolism

        patch = world.patches[self.position]
        self.eat(patch)
        self.move(world)

        # Размножение
        children = []
        if self.breed_type == 1:  # бесполое
            if random.random() < (self.chance_num / self.chance_den):
                for _ in range(self.birth_count):
                    children.append(self.clone(current_tick))
        elif self.breed_type == 2:  # половое
            for cow in world.cows:
                if cow != self and cow.position == self.position:
                    if random.random() < (self.chance_num / self.chance_den):
                        children.append(self.reproduce(cow, current_tick))
                    break

        world.new_cows.extend(children)
        return self.die(age)

    def die(self, age):
        T = self.age_min + self.age_mult * age
        p_death = self.age_t_weight * math.exp(T)
        if self.energy <= 0 or age >= self.T0 or random.random() < p_death:
            self.death_tick = self.birth_tick + age
            return True
        return False

    def clone(self, current_tick):
        child = Cow(
            position=self.position,
            birth_tick=current_tick,
            config=self.config,
            parents_qty=1
        )

        # мутации
        child.size = max(0.1, self.size + random.uniform(-0.05, 0.05))
        child.speed = max(1, self.speed + random.choice([-1, 0, 1]))
        child.metabolism = max(0.001, self.metabolism + random.uniform(-0.01, 0.01))
        child.sight = max(1, self.sight + random.choice([-1, 0, 1]))
        child.herding = max(1, self.herding + random.choice([-1, 0, 1]))
        return child

    def reproduce(self, partner, current_tick):
        num_offspring = random.randint(1, self.config["BIRTH_COUNT"])
        offspring_list = []
    
        for _ in range(num_offspring):
            child = Cow(
                position=self.position,
                birth_tick=current_tick,
                config=self.config,
                parents_qty=2
            )
    
            # среднее значение параметров от родителей + мутации
            child.size = (self.size + partner.size) / 2 + random.uniform(-0.05, 0.05)
            child.speed = max(1, int((self.speed + partner.speed) / 2 + random.choice([-1, 0, 1])))
            child.metabolism = (self.metabolism + partner.metabolism) / 2 + random.uniform(-0.01, 0.01)
            child.sight = max(1, int((self.sight + partner.sight) / 2 + random.choice([-1, 0, 1])))
            child.herding = max(1, int((self.herding + partner.herding) / 2 + random.choice([-1, 0, 1])))
    
            offspring_list.append(child)
    
        return offspring_list



In [77]:
class Simulation:
    def __init__(self, config_data):
        self.config_data = config_data
        self.world = World(config_data)  # Передаем весь конфиг в мир
        self.num_years = config_data["TIME"]
        self.ticks = self.num_years * config_data["SEASON_LENGTH"]
        self.current_tick = 0

        # Заполняем начальную популяцию коров
        self.cows = self.create_initial_cows()

        # Логирование
        self.log_file = open("simulation_log.txt", "w")
        self.log_initial_stats()

    def create_initial_cows(self):
        cows = []
        # Равномерно распределяем коров по всем клеткам мира
        for _ in range(self.config_data["COWS_COUNT"]):
            birth_tick = random.randint(0, self.ticks)
            
            position = random.randint(0, self.world.length - 1)
            
            cow = Cow(
                position=position,
                birth_tick=birth_tick,
                config=self.config_data  # Передаем конфиг в корову
            )
            cows.append(cow)
        return cows

    def log(self, message):
        self.log_file.write(f"{message}\n")

    def log_initial_stats(self):
        self.log("Initial Stats:")
        # Логирование начальных данных
        soil_count = {'desert': 0, 'steppe': 0, 'field': 0, 'forest': 0}
        initial_grass = 0.0

        for patch in self.world.patches:
            soil_count[patch.soil_quality] += 1
            initial_grass += patch.grass

        self.log(f"Initial distribution of soil types: {soil_count}")
        self.log(f"Total initial grass: {initial_grass}")

    def run(self):
        for tick in range(self.ticks):
            self.current_tick = tick
            self.world.update()
            new_cows = self.cows[:]
            self.cows.clear()

            for cow in new_cows:
                if not cow.update(self.current_tick, self.world):  # если не умерла
                    self.cows.append(cow)

            self.log(f"Tick {tick}: {len(self.cows)} cows alive")
            if len(self.cows) == 0:
                self.log("All cows are dead. Simulation stopped.")
                break

        self.log_file.close()


In [78]:
simulation = Simulation(config_data)

simulation.run()

print("Симуляция завершена. Логи сохранены в 'simulation_log.txt'.")


Симуляция завершена. Логи сохранены в 'simulation_log.txt'.
